In [1]:
import SALib
from SALib.sample import saltelli
from SALib.analyze import sobol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
from IPython.display import clear_output
#from tqdm import tqdm  # For progress bar
from ABM_Final_Final_Final import SocialNetwork

In [36]:
# We define our variables and bounds
problem = {
    'num_vars': 4,
    'names': ['prob', 'mu', 'temp', 'social'],
    'bounds': [[0., 1.], [0.5,2], [0,100], [0.,1.]]
}

w_pop = 0.2
w_prox = 0.7
w_sim = 0.1

# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 5
max_steps = 800
distinct_samples = 8

# We get all our samples here
param_values = saltelli.sample(problem, distinct_samples, calc_second_order=False)

# Initialize DataFrame to store results
num_samples = len(param_values) * replicates

data = pd.DataFrame(index=range(num_samples), columns=['prob', 'mu', 'temp', 'social'])

data['Run'], data['max IN degree'], data['avg clustering coeff'] = None, None, None

print(data)

   prob   mu temp social   Run max IN degree avg clustering coeff
0   NaN  NaN  NaN    NaN  None          None                 None
1   NaN  NaN  NaN    NaN  None          None                 None
2   NaN  NaN  NaN    NaN  None          None                 None
3   NaN  NaN  NaN    NaN  None          None                 None
4   NaN  NaN  NaN    NaN  None          None                 None
5   NaN  NaN  NaN    NaN  None          None                 None
6   NaN  NaN  NaN    NaN  None          None                 None
7   NaN  NaN  NaN    NaN  None          None                 None
8   NaN  NaN  NaN    NaN  None          None                 None
9   NaN  NaN  NaN    NaN  None          None                 None
10  NaN  NaN  NaN    NaN  None          None                 None
11  NaN  NaN  NaN    NaN  None          None                 None
12  NaN  NaN  NaN    NaN  None          None                 None
13  NaN  NaN  NaN    NaN  None          None                 None
14  NaN  N

C:\Users\franc\AppData\Local\Temp\ipykernel_16856\2109614244.py:18: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  param_values = saltelli.sample(problem, distinct_samples, calc_second_order=False)


In [37]:
# Define your model
def run_model(params,max_steps):
    n_agents = 100
    prob, temp, mu, social = params
    model = SocialNetwork(n_agents, prob, w_pop, w_prox, w_sim, mu, temp, social)
    results = pd.DataFrame(model.Data_Collector)
    for k in range(max_steps + 1):
        model.step()
        print(f"\rProgress: {(k / max_steps) * 100:.2f}%", end='', flush=True)
   
    df_results = pd.DataFrame(model.Data_Collector)
    
    return df_results['avg clustering coeff'].iloc[-1], df_results['max IN degrees'].iloc[-1]

In [38]:
# Batch run
count = 0
for i in range(replicates):
    for vals in param_values:
        
        # Run the model with the current set of parameters
        print(f"\nRun {count}")
        # iteration[0] is clustercoef, iteration[1] is max_IN
        clustering_coeff, max_in = run_model(vals, max_steps)
        data.iloc[count, 0:len(vals)] = list(vals)
        data.iloc[count, len(vals):len(vals)+3] = count,clustering_coeff,max_in
        count += 1
       
data.to_csv(f"sobol_rep{replicates}_maxstep{max_steps}_distsamples{distinct_samples}.csv", index=False)



Run 0
Progress: 100.00%
Run 1
Progress: 100.00%
Run 2
Progress: 100.00%
Run 3
Progress: 100.00%
Run 4
Progress: 100.00%
Run 5
Progress: 100.00%
Run 6
Progress: 100.00%
Run 7
Progress: 100.00%
Run 8
Progress: 100.00%
Run 9
Progress: 100.00%
Run 10
Progress: 100.00%
Run 11
Progress: 100.00%
Run 12
Progress: 100.00%
Run 13
Progress: 100.00%
Run 14
Progress: 100.00%
Run 15
Progress: 100.00%
Run 16
Progress: 100.00%
Run 17
Progress: 100.00%
Run 18
Progress: 100.00%
Run 19
Progress: 100.00%
Run 20
Progress: 100.00%
Run 21
Progress: 100.00%
Run 22
Progress: 100.00%
Run 23
Progress: 100.00%

In [41]:
Si_sheep = sobol.analyze(problem, data['avg clustering coeff'].values, calc_second_order=False, print_to_console=True)
Si_wolves = sobol.analyze(problem, data['max IN degrees'].values, calc_second_order=False, print_to_console=True)

              ST   ST_conf
prob    0.191684  1.092970
mu      0.004470  0.060527
temp    0.000000  0.000000
social  0.002407  0.019204
              S1   S1_conf
prob    0.128248  5.370529
mu     -0.062920  1.032934
temp    0.000000  0.000000
social -0.050199  0.644975


C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


KeyError: 'max IN degrees'

In [39]:
print(data)

       prob        mu    temp   social Run max IN degree avg clustering coeff
0   0.09375  1.203125  46.875  0.65625   0       0.10492                   18
1   0.28125  1.203125  46.875  0.65625   1      0.276065                   39
2   0.09375  1.953125  46.875  0.65625   2      0.102328                   19
3   0.09375  1.203125  53.125  0.65625   3      0.092677                   18
4   0.09375  1.203125  46.875  0.84375   4      0.098414                   21
5   0.28125  1.953125  53.125  0.84375   5        0.2781                   36
6   0.59375  1.953125  96.875  0.15625   6      0.591265                   72
7   0.78125  1.953125  96.875  0.15625   7      0.768627                   85
8   0.59375  1.203125  96.875  0.15625   8      0.585988                   72
9   0.59375  1.953125   3.125  0.15625   9      0.581127                   72
10  0.59375  1.953125  96.875  0.34375  10      0.592285                   71
11  0.78125  1.203125   3.125  0.34375  11      0.773077        